In [5]:
# ────────────────────────────────────────────────
# 1. CARGAR LIBRERIAS 
# ────────────────────────────────────────────────

import os
# → Para leer claves secretas del archivo .env (como la clave de Last.fm)

import time
# → Para hacer pausas cortas y no sobrecargar la API

import pandas as pd
# → Para guardar los resultados en una tabla (como Excel)

import requests
# → Para hacer peticiones a la API de Last.fm (no usa una librería especial como Spotipy)

from dotenv import load_dotenv
# → Para cargar el archivo .env donde guardas tu clave secreta

In [6]:
# ────────────────────────────────────────────────
# 2. CARGAR Y VALIDAR LA CLAVE DE LAST.FM
# ────────────────────────────────────────────────

# Carga el archivo .env (donde está tu clave de Last.fm)
load_dotenv('.env', override=True)

# Extrae tu clave de API (es pública, pero igual la guardamos en .env por orden)
mi_clave_lastfm = os.getenv('LASTFM_API_KEY')

# Verifica que exista
if not mi_clave_lastfm:
    raise ValueError(
        "Error: Falta tu clave de Last.fm en el archivo .env"
        "Asegúrate de tener una línea como:"
        "LASTFM_API_KEY=tu_clave_aquí"
    )

print("Conexión lista para usar Last.fm")

Conexión lista para usar Last.fm


In [9]:
# ─── 1. Leer tu archivo de Spotify ───────────────────────────────
print("Leyendo artistas desde tu archivo de Spotify...")
spotify_df = pd.read_csv("spotify_canciones.csv")

# Sacamos solo los nombres de los artistas, sin repetir
artistas_unicos = spotify_df["artista"].drop_duplicates().tolist()
print(f"Encontré {len(artistas_unicos)} artistas diferentes.\n")

# ─── 2. Preparar una lista vacía para guardar lo que encontremos ───
datos_de_lastfm = []

# ─── 3. Preguntar a Last.fm por cada artista ─────────────────────
print("Buscando información en Last.fm...")

for artista in artistas_unicos:
    print(f"  → Buscando: {artista}")
    
    # Hacemos la petición a la API de Last.fm
    respuesta = requests.get(
        "http://ws.audioscrobbler.com/2.0/",
        params={
            "method": "artist.getinfo",
            "artist": artista,
            "api_key": clave,
            "format": "json"
        }
    )
    
    # Si la conexión falla (ej. error de internet o servidor)
    if respuesta.status_code != 200:
        print(f"    ❌ Error de conexión para '{artista}'")
        continue  # Saltamos al siguiente artista
    
    # Convertimos la respuesta en JSON para poder usarla
    datos = respuesta.json()
    
    # Si Last.fm no conoce al artista
    if "error" in datos:
        print(f"    ⚠️  '{artista}' no encontrado en Last.fm")
        continue  # Saltamos al siguiente artista

    # ─── 4. Extraer la información que nos interesa ───────────
    
    # ------------------ Biografía ------------------
    # Primero comprobamos si la clave 'bio' existe en los datos
    if "bio" in datos["artist"]:
        # Si existe, obtenemos el contenido de la biografía
        bio = datos["artist"]["bio"]["content"]
    else:
        # Si no existe, ponemos un texto por defecto
        bio = "Sin biografía"

    # Quitamos el texto "Read more on Last.fm" si aparece
    if "Read more on Last.fm" in bio:
        bio = bio.split("Read more on Last.fm")[0].strip()

    # ------------------ Oyentes y reproducciones ------------------
    # Obtenemos el número de oyentes y reproducciones
    # Si no existen, ponemos 'desconocido'
    if "stats" in datos["artist"]:
        oyentes = datos["artist"]["stats"].get("listeners", "desconocido")
        reproducciones = datos["artist"]["stats"].get("playcount", "desconocido")
    else:
        oyentes = "desconocido"
        reproducciones = "desconocido"

    # ------------------ Guardar los datos ------------------
    datos_de_lastfm.append({
        "artista": artista,
        "biografia": bio,
        "oyentes": oyentes,
        "reproducciones": reproducciones
    })
    
    # Esperamos un poco para no hacer demasiadas peticiones seguidas
    time.sleep(0.25)

# ─── 5. Guardar los datos de Last.fm en su propio archivo ─────────
if datos_de_lastfm:
    df_lastfm = pd.DataFrame(datos_de_lastfm)
    df_lastfm.to_csv("lastfm_artistas.csv", index=False, encoding="utf-8")
    print(f"\n✅ ¡Listo! Se guardó el archivo: lastfm_artistas.csv")
    print(f"Contiene información de {len(df_lastfm)} artistas.")
else:
    print("\n⚠️  No se encontró información de ningún artista en Last.fm.")
    print("El archivo lastfm_artistas.csv no se creó.")

print("\n💡 Recuerda: este archivo SOLO tiene datos de Last.fm.")
print("Después puedes unirlo con tu archivo de Spotify usando la columna 'artista'.")


Leyendo artistas desde tu archivo de Spotify...


FileNotFoundError: [Errno 2] No such file or directory: 'spotify_canciones.csv'